In [1]:
# Original Dataset: https://www.cs.toronto.edu/~kriz/cifar.html for more information
# Load of necessary libraries
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.layers.convolutional import Conv2D
from tensorflow.keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# to make the example replicable
np.random.seed(42)
# Load of the dataset
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

In [2]:
X_train.shape, Y_train.shape

((50000, 32, 32, 3), (50000, 1))

### reduce dataset size

In [3]:
df = pd.DataFrame(list(zip(X_train, Y_train)), columns =['Image', 'Label']) 
val = df.sample(frac=(0.02))
# X_train = np.array([ i for i in list(val['Image'])])
# Y_train = np.array([ [i[0]] for i in list(val['Label'])])
X = np.array([ i for i in list(val['Image'])])
Y = np.array([ [i[0]] for i in list(val['Label'])])

In [4]:
X.shape, Y.shape# X_train.shape, Y_train.shape

((1000, 32, 32, 3), (1000, 1))

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

## Normalization

In [7]:
# Centering the data
X_train_mean = np.mean(X_train, axis = 0)
X_train_cent = X_train - X_train_mean
# Normalization
X_train_std = np.std(X_train, axis = 0)
X_train_norm = X_train_cent / X_train_std

## Train/Test

In [8]:
# preparing test set
X_test_norm = (X_test - X_train_mean) / X_train_std

## Conv Model

In [17]:
from keras.layers import BatchNormalization, Activation
from keras.regularizers import l1, l2, l1_l2
from keras.constraints import max_norm


class Convolutional:
    
    def __init__(self, desc, batch_size=128, shuffle=True, epochs=10, batch_norm=False, 
                 k_regularizer=None, k_constraint=None, dropout=False, 
                 dropout_val=0.5, max_pooling=False):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.epochs = epochs
        self.desc = desc
        self.scores = []
        
        if max_pooling and dropout:
            print('why')
            return None
        
        # Inizializting the model
        self.model = Sequential()
        # Defining a convolutional layer
        self.model.add(Conv2D(128, kernel_size=(3, 3), input_shape=(32, 32, 3)))
        if batch_norm: self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))
        if max_pooling: self.model.add(MaxPooling2D(
            pool_size=(2, 2), strides=strides, padding="valid", data_format=None
        ))
        if dropout: self.model.add(Dropout(dropout_val/2))

        # Defining a second convolutional layer
        self.model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
        if batch_norm: self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))
        if max_pooling: self.model.add(MaxPooling2D(
            pool_size=(2, 2), strides=strides, padding="valid", data_format=None,
        ))
        if dropout: self.model.add(Dropout(dropout_val/2))

        # Defining a third convolutional layer
        self.model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
        if batch_norm: self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))
        if max_pooling: self.model.add(MaxPooling2D(
            pool_size=(2, 2), strides=strides, padding="valid", data_format=None,
        ))
        if dropout: self.model.add(Dropout(dropout_val/2))

        # We include our classifier
        self.model.add(Flatten())
        self.model.add(Dense(1024, activation='relu', kernel_regularizer=k_regularizer, kernel_constraint=k_constraint))
        if dropout: self.model.add(Dropout(dropout_val))
        self.model.add(Dense(10, activation='softmax'))
        
        # Compiling the model
        self.model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(learning_rate=0.0001, decay=1e-6),
                      metrics=['accuracy'])
        
    def train(self, X_train, Y_train, X_test, Y_test):
        # Training of the model
        self.model.fit(X_train, to_categorical(Y_train),
                  batch_size=self.batch_size,
                  shuffle=self.shuffle,
                  epochs=self.epochs,
                  validation_data=(X_test, to_categorical(Y_test)))
        
        # Evaluation of the model
        self.scores = self.model.evaluate(X_test, to_categorical(Y_test))
        return self.scores
        
    def __str__(self):
        return self.desc+":\n\t Loss: "+ str(self.scores[0]) +"\t Accuracy: "+ str(self.scores[1])
     

## Instantiate models & fit

In [12]:
from keras.layers import BatchNormalization, Activation
from keras.regularizers import l1, l2, l1_l2
from keras.constraints import max_norm
from keras.layers import Dropout


models = []

batch_size = 256
epochs = 10
strides=1

## Batch Norm
models.append(Convolutional(desc="Batch_Norm", batch_size=batch_size, epochs=epochs, \
                             batch_norm=True))

## Regularization
models.append(Convolutional(desc="Regularization_l1_10^-2", \
                            batch_size=batch_size, epochs=epochs, \
                            k_regularizer=l1(10**-2)))
models.append(Convolutional(desc="Regularization_l2_10^-2", \
                            batch_size=batch_size, epochs=epochs, \
                            k_regularizer=l2(10**-2)))
models.append(Convolutional(desc="Regularization_elastic_10^-2", \
                            batch_size=batch_size, epochs=epochs, \
                            k_regularizer=l1_l2(10**-2)))
    
## Max Norm & Dropout
models.append(Convolutional(desc="Max_Norm", \
                            batch_size=batch_size, epochs=epochs, \
                            k_constraint=max_norm(3. )))

models.append(Convolutional(desc="Dropout", \
                            batch_size=batch_size, epochs=epochs, \
                            dropout = True))

models.append(Convolutional(desc="Max_Norm_And_Dropout", \
                            batch_size=batch_size, epochs=epochs, \
                            k_constraint=max_norm(3. ), dropout=True))

## Max Pooling
models.append(Convolutional(desc="Max_Pooling", \
                            batch_size=batch_size, epochs=epochs, \
                            max_pooling=True))

## Fit models & Report scores

In [13]:
for m in models:
    m.train(X_train_norm, Y_train, X_test_norm, Y_test)
    print(m)

Epoch 1/10
4/4 [==============================] - 11s 2s/step - loss: 11.4644 - accuracy: 0.1363 - val_loss: 2.7169 - val_accuracy: 0.1000
Epoch 2/10
4/4 [==============================] - 10s 2s/step - loss: 8.9772 - accuracy: 0.2725 - val_loss: 2.3022 - val_accuracy: 0.1400
Epoch 3/10
4/4 [==============================] - 10s 2s/step - loss: 2.7966 - accuracy: 0.4563 - val_loss: 2.2733 - val_accuracy: 0.1700
Epoch 4/10
4/4 [==============================] - 10s 2s/step - loss: 1.3378 - accuracy: 0.5437 - val_loss: 2.2672 - val_accuracy: 0.2600
Epoch 5/10
4/4 [==============================] - 10s 2s/step - loss: 0.9315 - accuracy: 0.7450 - val_loss: 2.2620 - val_accuracy: 0.2750
Epoch 6/10
4/4 [==============================] - 10s 2s/step - loss: 0.5841 - accuracy: 0.8737 - val_loss: 2.2645 - val_accuracy: 0.2600
Epoch 7/10
4/4 [==============================] - 10s 2s/step - loss: 0.3757 - accuracy: 0.9438 - val_loss: 2.2672 - val_accuracy: 0.2650
Epoch 8/10
4/4 [=================

In [20]:
for m in sorted(models, key=lambda m: m.scores[1]):
    print(m)

Regularization_elastic_10^-2:	 Loss: 1028.919189453125	 Accuracy: 0.23499999940395355
Batch_Norm:	 Loss: 2.2706074714660645	 Accuracy: 0.2549999952316284
Regularization_l1_10^-2:	 Loss: 1024.9365234375	 Accuracy: 0.2849999964237213
Max_Norm_And_Dropout:	 Loss: 1.8752881288528442	 Accuracy: 0.3400000035762787
Dropout:	 Loss: 1.8582043647766113	 Accuracy: 0.3449999988079071
Regularization_l2_10^-2:	 Loss: 9.117854118347168	 Accuracy: 0.3499999940395355
Max_Pooling:	 Loss: 1.7532321214675903	 Accuracy: 0.4050000011920929
Max_Norm:	 Loss: 1.956183671951294	 Accuracy: 0.4350000023841858


In [21]:
from keras.layers import BatchNormalization, Activation
from keras.regularizers import l1, l2, l1_l2
from keras.constraints import max_norm
from keras.layers import Dropout


models2 = []

batch_size = 256
epochs = 10
strides=1

## Batch Norm
models2.append(Convolutional(desc="Batch_Norm", batch_size=batch_size, epochs=epochs, \
                             batch_norm=True))

## Regularization
models2.append(Convolutional(desc="Regularization_l1_10^-2", \
                            batch_size=batch_size, epochs=epochs, batch_norm=True, \
                            k_regularizer=l1(10**-2)))
models2.append(Convolutional(desc="Regularization_l2_10^-2", \
                            batch_size=batch_size, epochs=epochs, batch_norm=True, \
                            k_regularizer=l2(10**-2)))
models2.append(Convolutional(desc="Regularization_elastic_10^-2", \
                            batch_size=batch_size, epochs=epochs, batch_norm=True, \
                            k_regularizer=l1_l2(10**-2)))
    
## Max Norm & Dropout
models2.append(Convolutional(desc="Max_Norm", \
                            batch_size=batch_size, epochs=epochs, batch_norm=True, \
                            k_constraint=max_norm(3. )))

models2.append(Convolutional(desc="Dropout", \
                            batch_size=batch_size, epochs=epochs, batch_norm=True, \
                            dropout = True))

models2.append(Convolutional(desc="Max_Norm_And_Dropout", \
                            batch_size=batch_size, epochs=epochs, batch_norm=True, \
                            k_constraint=max_norm(3. ), dropout=True))

## Max Pooling
models2.append(Convolutional(desc="Max_Pooling", \
                            batch_size=batch_size, epochs=epochs, batch_norm=True, \
                            max_pooling=True))

In [22]:
for m in models2:
    m.train(X_train_norm, Y_train, X_test_norm, Y_test)
    print(m)

Epoch 1/10
4/4 [==============================] - 11s 2s/step - loss: 9.1297 - accuracy: 0.1375 - val_loss: 2.4204 - val_accuracy: 0.1500
Epoch 2/10
4/4 [==============================] - 10s 2s/step - loss: 6.6822 - accuracy: 0.2812 - val_loss: 2.2775 - val_accuracy: 0.1550
Epoch 3/10
4/4 [==============================] - 10s 2s/step - loss: 3.0213 - accuracy: 0.4112 - val_loss: 2.2732 - val_accuracy: 0.2850
Epoch 4/10
4/4 [==============================] - 11s 3s/step - loss: 1.2217 - accuracy: 0.6662 - val_loss: 2.2789 - val_accuracy: 0.2750
Epoch 5/10
4/4 [==============================] - 13s 3s/step - loss: 0.8361 - accuracy: 0.7538 - val_loss: 2.2784 - val_accuracy: 0.3100
Epoch 6/10
4/4 [==============================] - 13s 3s/step - loss: 0.5009 - accuracy: 0.9275 - val_loss: 2.2770 - val_accuracy: 0.3250
Epoch 7/10
4/4 [==============================] - 17s 4s/step - loss: 0.2972 - accuracy: 0.9700 - val_loss: 2.2759 - val_accuracy: 0.3250
Epoch 8/10
4/4 [==================

In [24]:
for m in sorted(models2, key=lambda m: m.scores[1]):
    print(m)

Regularization_l1_10^-2:
	 Loss: 1077.66064453125	 Accuracy: 0.17499999701976776
Regularization_elastic_10^-2:
	 Loss: 1073.2086181640625	 Accuracy: 0.23499999940395355
Max_Norm_And_Dropout:
	 Loss: 2.290571689605713	 Accuracy: 0.23499999940395355
Max_Norm:
	 Loss: 2.2770023345947266	 Accuracy: 0.24500000476837158
Dropout:
	 Loss: 2.2661662101745605	 Accuracy: 0.28999999165534973
Max_Pooling:
	 Loss: 2.2657642364501953	 Accuracy: 0.28999999165534973
Batch_Norm:
	 Loss: 2.27367901802063	 Accuracy: 0.3149999976158142
Regularization_l2_10^-2:
	 Loss: 19.809864044189453	 Accuracy: 0.3199999928474426
